In [ ]:
# 1) Instalação (se ainda não tiver)
%pip install ipywidgets ipython-genutils --quiet

# 2) Imports
import re, json, os
from IPython.display import display, clear_output
import ipywidgets as widgets
import ipywidgets
from ipywidgets import Layout


# 3) Campos de texto com validação
nome = widgets.Text(description="Nome:")
email = widgets.Text(description="Email:")
cpf = widgets.Text(description="CPF:")
twitter = widgets.Text(
    description="Twitter/X:",
    placeholder="https://twitter.com/seu_usuario"
)

# 4) Outros widgets
interesses = widgets.SelectMultiple(
    options=[
        "Jogos", "Merchandising", "Eventos presenciais",
        "Twitch/Streaming", "Campeonatos online", "Cosplay",
        "Comunidades Discord", "Conteúdo no YouTube"
    ],
    description="Interesses:",
    rows=6,
    style={"description_width": "initial"},
)

descricao = widgets.Textarea(
    placeholder="Conte aqui um pouco sobre você, o fã de e-sports…",
    description="Descrição:",
    layout=Layout(width="100%", height="100px"),  # Layout agora importado
    style={"description_width": "initial"},
)

btn = widgets.Button(description="Enviar", button_style="primary")
out = widgets.Output()

DATA_FILE = 'fans_data.jsonl'

# 5) Callback
def on_submit(b):
    with out:
        clear_output()
        if not re.match(r'^[A-Za-zÀ-ÖØ-öø-ÿ ]+$', nome.value):
            print("❌ Nome inválido! Apenas letras e espaços.")
            return
        if not re.match(r'^[^@]+@[^@]+\.com$', email.value):
            print("❌ Email inválido! Formato: nome@provedor.com")
            return
        if not cpf.value.isdigit():
            print("❌ CPF inválido! Apenas dígitos.")
            return

        record = {
            "nome": nome.value,
            "email": email.value,
            "cpf": cpf.value,
            "twitter": twitter.value,
            "interesses": list(interesses.value),
            "descricao": descricao.value
        }
        with open(DATA_FILE, "a", encoding="utf-8") as f:
            json.dump(record, f, ensure_ascii=False)
            f.write("\n")

        print("✅ Gravado:", record)

btn.on_click(on_submit)

# 6) Exibe o formulário
display(nome, email, cpf, twitter, interesses, descricao, btn, out)


Note: you may need to restart the kernel to use updated packages.


Text(value='', description='Nome:')

Text(value='', description='Email:')

Text(value='', description='CPF:')

Text(value='', description='Twitter/X:', placeholder='https://twitter.com/seu_usuario')

SelectMultiple(description='Interesses:', options=('Jogos', 'Merchandising', 'Eventos presenciais', 'Twitch/St…

Textarea(value='', description='Descrição:', layout=Layout(height='100px', width='100%'), placeholder='Conte a…

Button(button_style='primary', description='Enviar', style=ButtonStyle())

Output()

Verificar o banco de dados.

In [6]:
import json

with open("fans_data.jsonl", encoding="utf-8") as f:
    registros = [json.loads(linha) for linha in f]
print("Registros carregados:") 
print(registros)
print("Total de registros:", len(registros))

Registros carregados:
[{'nome': 'João Victor de Almeida', 'email': 'victoralmeida71@hotmail.com', 'cpf': '47498873826', 'twitter': 'x.com/JottaAlmeidZ', 'interesses': ['Twitch/Streaming'], 'descricao': 'Gosto muito de esports, principalmente jogos como LoL e CS2. Apaixonado por games'}]
Total de registros: 1


Esvaziar os Dados dos usuários

In [ ]:
# 1) Certifique-se de usar o mesmo caminho do arquivo
DATA_FILE = 'fans_data.jsonl'  # ou '/content/fans_data.jsonl' se quiser ser explícito

# 2) Abra em modo escrita e imediatamente feche — isso apaga tudo dentro
with open(DATA_FILE, 'w', encoding='utf-8'):
    pass

print(f"O arquivo {DATA_FILE} foi esvaziado.")

Ronhecimento da foto, com meus dados da foto

In [7]:
# 1) Instale os pacotes necessários (se ainda não tiver)
%pip install pytesseract face_recognition ipywidgets --quiet

# 2) Imports
import re, json, io, os
from IPython.display import display, clear_output
import pytesseract, face_recognition
from PIL import Image
import ipywidgets as widgets

# 3) Carrega registros existentes
DATA_FILE = "fans_data.jsonl"
records = []
if os.path.exists(DATA_FILE):
    with open(DATA_FILE, "r", encoding="utf-8") as f:
        for line in f:
            try:
                records.append(json.loads(line))
            except:
                pass


upload_self  = widgets.FileUpload(accept='image/*', description="Selfie")
btn_verify   = widgets.Button(description="Verificar & Salvar", button_style="primary")
out          = widgets.Output(layout=widgets.Layout(border='1px solid gray'))



def on_verify(b):
    with out:
        clear_output()
        cpf = cpf_input.value.strip()
        if not upload_doc.value or not upload_self.value:
            print("❌ Faça upload da foto do documento E da selfie.")
            return
        doc_bytes  = list(upload_doc.value.values())[0]['content']
        self_bytes = list(upload_self.value.values())[0]['content']

        # OCR no documento (opcional, só pra conferência)
        doc_text = pytesseract.image_to_string(Image.open(io.BytesIO(doc_bytes)))
        print("🔍 Texto extraído do documento (para conferência):")
        print(doc_text.strip() or "– nada extraído –")

        # Encodings faciais
        enc_doc  = face_recognition.face_encodings(face_recognition.load_image_file(io.BytesIO(doc_bytes)))
        enc_self = face_recognition.face_encodings(face_recognition.load_image_file(io.BytesIO(self_bytes)))
        if not enc_doc or not enc_self:
            print("❌ Não detectei rosto em uma das imagens.")
            return

        match = face_recognition.compare_faces([enc_doc[0]], enc_self[0])[0]
        print(f"🤖 Reconhecimento facial: {'👍 Mesma pessoa' if match else '❌ Não corresponde'}")

        # Salva encoding da selfie no registro
        rec["face_encoding"] = enc_self[0].tolist()
        # Reescreve todo o arquivo JSONL
        with open(DATA_FILE, "w", encoding="utf-8") as f:
            for r in records:
                f.write(json.dumps(r, ensure_ascii=False) + "\n")

        print(f"✅ Face encoding salvo para CPF {cpf}")

# vincula callback
btn_verify.on_click(on_verify)

# 5) Exibe formulário
display(
    cpf_input,
    upload_doc,
    upload_self,
    btn_verify,
    out
)

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━╸━━━━━━━━━━━━━━━━━━ 52.4/100.1 MB 217.1 kB/s eta 0:03:40
error: incomplete-download

× Download failed because not enough bytes were received (52.4 MB/100.1 MB)
╰─> URL: https://files.pythonhosted.org/packages/cf/3b/4fd8c534f6c0d1b80ce0973d01331525538045084c73c153ee6df20224cf/face_recognition_models-0.3.0.tar.gz

note: This is an issue with network connectivity, not pip.
hint: Consider using --resume-retries to enable download resumption.
Note: you may need to restart the kernel to use updated packages.


ModuleNotFoundError: No module named 'pytesseract'

In [ ]:
import tweepy

auth = tweepy.OAuth1UserHandler(consumer_key, consumer_secret, callback="oob")
redirect_url = auth.get_authorization_url()
print("Abra no navegador:", redirect_url)

verifier = input("Cole o PIN aqui: ")
auth.get_access_token(verifier)
api = tweepy.API(auth)
tweets = api.user_timeline(count=20)
for t in tweets:
    print("-", t.text)


In [ ]:
import requests
from bs4 import BeautifulSoup
from openai import OpenAI

# Exemplo de scraping de perfil
resp = requests.get("https://liquipedia.net/counterstrike/FURIA")
soup = BeautifulSoup(resp.text, "html.parser")
print("Título da página:", soup.title.string)

# Prompt para validação
client = OpenAI(api_key="…")
res = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role":"system", "content":"Você é…"},
        {"role":"user",   "content":"Esse perfil evidencia Fan de esports FURIA? Responda sim ou não."}
    ]
)
print(res.choices[0].message.content)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')